# Import modules & packages

To be able tu use declarations and definitions of an existing module, you should use the *import* clause.

In [ ]:
#import module [as alias]

#example
import os

#### Exercices

Use the import command as above to import additional modules or packages for the future of this pipeline.

In [ ]:
#the plotting module matplotlib.pyplot as plt
import matplotlib.pyplot as plt

#the numpy package for scientific computing and also the great tool for manipulate arrays
import numpy as np

#the image processing package skimage and ndimage from scipy
import skimage
import scipy.ndimage as ndi

# Importing & Handling Image Data

**Background**

Images are essentially a matrix of numbers.   
Image processing means simply to carry out mathematical operations on these numbers.   
The ideal object for storing and manipulating matrix of numbers is the array.   
Many mathematical operations are well defined on arrays and can be computed quickly by vector-based computation.   
Arrays can have any number of dimensions (or "axis"). For example, a 2D array could represent the x and y axis of a grayscale image, a 3D array could contain a z-stack (zyx), a 4D array could also have multiple channels for each image (czyx) and a 5D array could have time on top of that (tczyx).

#### Exercices

Follow the instructions in the comments below.

In [ ]:
# (i) Specify the filename
# Create a string variable of the file to be imported ('FITC.jpeg')
# Suggested name for the variable: filename_1
# Note: If the file is not in your current working directory, the filename variable must contain the 
#       entire path to the file, for example r'/home/cedric/data/FITC.jpeg'. Note the r at
#       the beginning of the string: it designates this string as a "raw" string, which helps to
#       avoid problems with slashes and other special symbols
filename_1 = r'/home/cedric/Documents/Atelier_Mifobio/jupyter-workshop-mifobio-2018/exercices/PK-11B-pl1/Well-C003/FITC.jpeg'

In [ ]:
# Create a string variable of the file to be imported ('Hoechst.jpeg')
filename_2 = r'/home/cedric/Documents/Atelier_Mifobio/jupyter-workshop-mifobio-2018/exercices/PK-11B-pl1/Well-C003/Hoechst.jpeg'

# Create a string variable of the file to be imported ('Tritc.jpeg')
filename_3 = r'/home/cedric/Documents/Atelier_Mifobio/jupyter-workshop-mifobio-2018/exercices/PK-11B-pl1/Well-C003/Tritc.jpeg'

In [ ]:
# (ii) Load the images
# Import the function 'imread' from the module 'skimage.io'
from skimage.io import imread

# Load 'FITC.jpeg' and store it in a variable.
# Suggested name for the variable: fitc
fitc = imread(filename_1)

# Load 'Hoechst.jpeg' and store it in a variable.
# Suggested name for the variable: hoechst
hoechst = imread(filename_2)

# Load 'Tritc.jpeg' and store it in a variable.
# Suggested name for the variable: tritc
tritc = imread(filename_3)


In [ ]:
# (iii) Check the data represention of the images
# Check that 'fitc', 'hoechst', 'tritc' is a variable of type 'ndarray' - use Python's built-in function 'type'.
print("Fitc is of type:"),type(fitc)
print("Hoechst is of type:"),type(hoechst)
print("Tritc is of type:"),type(tritc)

# Print the shape of the array using the numpy-function 'shape'. 
print("Fitc has shape:"),fitc.shape
print("Hoechst has shape:"),hoechst.shape
print("Tritc has shape:"),tritc.shape

# Check the datatype of the individual numbers in the array. You can use the array attribute 'dtype' to do so.
print("Fitc values are of type:"),fitc.dtype
print("Hoechst values are of type:"),fitc.dtype
print("Tritc values are of type:"),tritc.dtype

In [ ]:
#(iv) Normalize images
#To compare image between them we can use numpy.iinfo function that get the range for n bits and 
# then divide the data by the max range to normalize image.

info = np.iinfo(hoechst.dtype)
hoechst = hoechst.astype(np.float64) / info.max

info = np.iinfo(fitc.dtype)
fitc = fitc.astype(np.float64) / info.max

info = np.iinfo(tritc.dtype)
tritc = tritc.astype(np.float64) / info.max

In [ ]:
# (v) Look at the images to check that all is OK.
# Use pyplot's functions plt.imshow followed by plt.show. 
# Check the documentation for plt.imshow and note the parameters that can be specified, such as the color map (cmap)
# and interpolation. Since you are working with scientific data, interpolation is unwelcome, so you should set it to
# 'none'. The most common cmap for grayscale images is naturally 'gray'.
# You may also want to adjust the size of the figure. You can do this by preparing the figure canvas with
# the function plt.figure before calling plt.imshow. The canvas size is adjusted using the keyword argument
# figsize when calling plt.figure.
plt.figure(figsize=(14,14))
plt.imshow(fitc,interpolation='none',cmap='gray')
plt.show()
plt.figure(figsize=(14,14))
plt.imshow(hoechst,interpolation='none',cmap='gray')
plt.show()
plt.figure(figsize=(14,14))
plt.imshow(tritc,interpolation='none',cmap='gray')
plt.show()

# (v) Create a composite image.
# First get a colored image by using the module gray2rgb from skimage.color.
# Apply different colors to the three channels. A green channel to tritc image, a blue channel to hoechst image, 
# a red channel to fitc image
# You can visualize the different channels in the same image by merging them; Merging channels by using the operator + 
# and so create a new variable composite as example. Then use the functions used above 
# to display the creating image. 
from skimage.color import gray2rgb

fitc_rgb = gray2rgb(fitc)
hoechst_rgb = gray2rgb(hoechst)
tritc_rgb = gray2rgb(tritc)

tritc_green = tritc_rgb * [0,1,0]
hoechst_blue = hoechst_rgb * [0,0,1]
fitc_red = fitc_rgb * [1,0,0]

composite = tritc_green + hoechst_blue + fitc_red
plt.figure(figsize=(14,14))
plt.imshow(composite,interpolation='none',cmap='gray')
plt.show()

composite.shape

# Computing nuclei mask

## Preprocessing

**Background**

The goal of image preprocessing is to prepare or optimize the images to make further analysis easier. The specific preprocessing steps used in a pipeline depend on the type of image, the microscopy technique used, the image quality, and the desired downstream analysis.   
In this pipeline we will be using a median filter then a background substraction. 

### Median filtering

Median filtering is useful as a smoothing technique for reduce noice.   
The median filter considers each pixel in the image in turn and looks at its nearby neighbors to decide whether or not it is representative of its surroundings. Instead of simply replacing the pixel value with the mean of neighboring pixel values, it replaces it with the median of those values. The median is calculated by first sorting all the pixel values from the surrounding neighborhood into numerical order and then replacing the pixel being considered with the middle pixel value.

#### Exercices

Follow the instructions in the comments below.

In [ ]:
# (i) Create a variable for the size of smoothing filter. 
#Either the sizes of a rectangular kernel or the footprint of the kernel must be provided. 
#The size parameter, if provided, must be a sequence of sizes or a single number in which case the size of 
#the filter is assumed to be equal along each axis. 
#The footprint if provided, must be an array that defines the shape of the kernel by its non-zero elements.
size = 5

In [ ]:
# (ii) Perform the smoothing on the image
# To do so, use the median filter function 'ndi.filters.median_filter' from the 
# image processing package ndimage, which was imported at the start of the tutorial. 
# Check out the documentation of scipy to see how to implement this function. 
# Allocate the output to a new variable.
hoechst_smooth = ndi.filters.median_filter(hoechst,size)

In [ ]:
# (iii) Visualize the result using plt.imshow and plt.show
plt.figure(figsize=(14,14))
plt.imshow(hoechst_smooth,interpolation='none',cmap='gray')
plt.show()

### Global Thresholding

Otsu's method is used to perform clustering based image thresholding. 
Assuming a bi-modal intensity distribution, pixels are separate into foreground and backround. 
The optimal threshold value to separate the two classes is determined by minimizing the combined intra-class variance 
or by maximizing the combined inter-class variance.

### Exercices

Follow the instructions in the comments below.

In [ ]:
#(i) Threshold the median-smoothed original image using the otsu method to obtain the nuclei mask.
# Set pixels with higher values in the original than in the bg to 1 and pixels with lower values to 0. 
# You can use a "relational operator" to do this, since numpy arrays will automatically perform element-wise
# comparisons when compared to other arrays of the same shape.
from skimage.filters import threshold_otsu

nmask = np.zeros(hoechst_smooth.shape, dtype = bool)
nmask[hoechst_smooth > threshold_otsu(hoechst_smooth)] = 1

# (ii) Visualize the result using plt.imshow and plt.show
plt.figure(figsize=(14,14))
plt.imshow(nmask, cmap = 'gray')
plt.show()

### Improving Masks with Binary Morphology

**Background**

Morphological operations such as erosion, dilation, closing and opening are common tools used (among other things) to improve masks after they are generated by thresholding.   
They can be used to fill small holes, remove noise, increase or decrease the size of an object, or smoothen mask outlines.   
Most morphological operations are - once again - simple kernel functions that are applied at each pixel of the image based on their neighborhood as defined by a structuring element (SE). For example, dilation simply assigns to the central pixel the maximum pixel value within the neighborhood; it is a maximum filter. Conversely, erosion is a minimum filter. Additional options emerge from combining the two: morphological closing, for example, is a dilation followed by an erosion. This is used to fill in gaps and holes or smoothing mask outlines without significantly changing the mask's area.   
Finally, there are also some more complicated morphological operations, such as hole filling.

#### Exercices

Follow the instructions in the comments below.

In [ ]:
# (i) Create a diamond-shaped structuring element and asign it to a new variable.
# Structuring elements are small binary images that indicate which pixels 
# should be considered as the 'neighborhood' of the central pixel.
# You can use skimage.morphology.diamond to create diamond-shaped structuring element

from skimage.morphology import diamond
se = diamond(1)

In [ ]:
# (ii) Try morphological operations to further improve the membrane mask
# The various operations are available in skimage, for example skimage.morphology.closing and ndimage for hole filling

from skimage.morphology import closing
from skimage.morphology import opening

nmask = opening(closing(nmask, se),se)
nmask = ndi.binary_fill_holes(nmask)

In [ ]:
# (iii) Visualize the result using plt.imshow and plt.show
plt.figure(figsize=(14,14))
plt.imshow(nmask, cmap = 'gray')
plt.show()

# Computing cell mask

#### Exercices

In [ ]:
# (i) Adjust Brightness and Contrast
# Before compute cell mask, try to increase the brigtness and the contrast of the image. 
# Note: Play with mathematical operator
# operator '+' or '-' for the brigthness
# operator '*' for contrast

ntritc = tritc * 2.82 - 0.17
nfitc = fitc * 5.03 - 0.35
nhoechst = hoechst * 2.99 - 0.15

#(ii) Visualize the result after create a mix image using plt.imshow and plt.show
mix = nfitc+ntritc+nhoechst
plt.figure(figsize=(14,14))
plt.imshow(mix, cmap = 'gray')
plt.show()

In [ ]:
#(iii) Do you improve the light of the image by playing with the gamma.
# test operator ^ (** in python) to apply gamma correction
mix_2 = (nfitc**2+ntritc**2+nhoechst**2)**0.5

#(iv) Visualize the result
plt.figure(figsize=(14,14))
plt.imshow(mix_2, cmap = 'gray')
plt.show()

### 2d convolution filter

Linear filtering is useful to perform low-pass filtering (to blur images, remove noise...) and high-pass filtering (to detect edges, sharpen images).

#### Exercices

In [ ]:
# (i) Create a cross-shaped structuring element and asign it to a new variable.
# Structuring elements are small binary images that indicate which pixels 
# should be considered as the 'neighborhood' of the central pixel.
# create a matrix with 3 columns and 3 rows as below
# 0 1 0
# 1 2 1
# 0 1 0
# then divide all values by 6 to obtain a float type matrix. Use the function matrix from numpy. 
se = np.matrix([[0,1,0],[1,2,1],[0,1,0]], dtype='float')/6

# (ii) import signal from scipy to get the convolve2d function for execute the 2d convolution filter on the image. 
# Read the documentation to get more details on the parameters of the function.
from scipy import signal

cmask = signal.convolve2d(mix_2, se, boundary="wrap", mode="same") >= 0.12

In [ ]:
#(iii) display the result
plt.imshow(cmask,interpolation='none',cmap='gray')
plt.show()

In [ ]:
# (iii) Create a diamond-shaped structuring element and asign it to a new variable.
# Structuring elements are small binary images that indicate which pixels 
# should be considered as the 'neighborhood' of the central pixel.
# You can use skimage.morphology.diamond to create diamond-shaped structuring element
se = diamond(1)

# (iv) Use morphology operation like closing to improve image
cmask = closing(cmask, se)

plt.imshow(cmask)

# segmenting nuclei using watershed

**Background**

The algorithm identifies and separates objects that stand out of the background (zero). It inverts the image and uses water to fill the resulting valleys (pixels with high intensity in the source image) until another object or background is met. The deepest valleys become indexed first, starting from 1.   
An array of seeds contains a few pixels at the center of each cell labeled by a unique ID number and otherwise surrounded by zeros. The expansion algorithm will start from these central pixels and grow outward until all zeros are overwritten by an ID label. In the case of watershed expansion, one can imagine the seeds as the sources from which water pours into the cells and starts filling them up.   
Here, we will use a distance transform and local maxima for seeding.

#### Exercices

In [ ]:
# (i) Distance transform on thresholded membranes
# Use the function ndi.distance_transform_edt.
dist_transf = ndi.distance_transform_edt(nmask)

In [ ]:
# (ii) Visualize the output and understand what you are seeing.
plt.figure(figsize=(7,7))
plt.imshow(dist_transf,interpolation='none',cmap='viridis')
plt.show()

In [ ]:
# (iii) Dilate the distance threshold
# Use ndi.filters.maximum_filter to dilate the distance transform.
i = 10
struct = (np.mgrid[:i,:i][0] - np.floor(i/2))**2 + (np.mgrid[:i,:i][1] - np.floor(i/2))**2 <= np.floor(i/2)**2
dist_trans_dil = ndi.filters.maximum_filter(dist_transf, footprint=struct) 
plt.figure(figsize=(7,7))
plt.imshow(dist_trans_dil, interpolation='none', cmap='viridis')
plt.show()

In [ ]:
# (iv) Retrieve the local maxima (the 'peaks') in the distance transform
# Use the function peak_local_max from the module skimage.feature.
from skimage.feature import peak_local_max
seeds = peak_local_max(dist_trans_dil, indices=False, min_distance=10)

In [ ]:
# (v) Visualize the output
# you can first plot the original input (or the smoothed) image and
# then plot the seeds on top of it before showing both with 'plt.show'.
plt.figure(figsize=(7,7))
plt.imshow(hoechst_smooth, interpolation='none', cmap='gray')
plt.imshow(np.ma.array(seeds,mask=seeds==0),interpolation='none',cmap='autumn')
plt.show()

In [ ]:
# (vi) Label the seeds
# Use connected component labeling to give each cell seed a unique ID number.
seeds_labeled = ndi.label(seeds)[0]

In [ ]:
# (vii) Perform watershed
# Use the function watershed from the module skimage.morphology.
# Use the labeled nuclei seeds and the smoothed nuclei image as input.
from skimage.morphology import watershed
ws = watershed(hoechst_smooth,seeds_labeled)

In [ ]:
# (viii) Show the result as transparent overlay over the smoothed input image
plt.figure(figsize=(7,7))
plt.imshow(hoechst_smooth, interpolation='none', cmap='gray')
plt.imshow(ws,interpolation='none',cmap='prism',alpha=0.4)
plt.show()

# segmenting cells using Voronoi tesselation

Find boundaries between adjacent regions in an image, where seeds have been already identified in the individual regions to be segmented. The method finds the Voronoi region of each seed on a manifold with a metric controlled by local image properties. The method is motivated by the problem of finding the borders of cells in microscopy images, given a labelling of the nuclei in the images.

In [ ]:
# (i) The label function from skimage.measure module finds every connected set of pixels other than the background, 
# and relabels these sets with a unique increasing integer.
from skimage.measure import label
markers_nuc = label(nmask, connectivity=2, return_num=True)

In [ ]:
# (ii) We apply a sobel filter from skimage.filters to detect edges from cell body and serve as input 
# for the voronoi tesselation algorithm. 
from skimage.filters import sobel

fsobel = np.empty_like(mix_2)
fsobel = sobel(mix_2)

# (iii) Visualize the output
plt.imshow(fsobel)

In [ ]:
# (iv) This algo is implemented in centrosome package, an open source image processing library.
from centrosome.propagate import propagate
cell_seg,_ = propagate(image=fsobel, labels=markers_nuc[0], mask=cmask, weight=1)

# (v) Visualize the output. Focus on boundaries between labeled 
# regions highlighted thanks to mark_boundaries from skimage.segmentation and draw contour lines 
# for nuclei thanks to contour from matplotlib.pyplot.
from skimage.segmentation import mark_boundaries
plt.imshow(mark_boundaries(composite, cell_seg))
plt.contour(nmask, colors = 'r', linewidths = 0.1)

# Quantification of cell features

**background**

The ultimate goal of image segmentation is of course the extraction of quantitative measurements, in this case on a single-cell level. Measures of interest can be based on intensity (in different channels) or on the size and shape of the cells.

we will extract the following:

    Cell ID (so all other measurements can be traced back to the cell that was measured)
    Mean intensity of each cell, for each channel
    The cell area i.e. the number of pixels that make up the cell

#### Exercices

Follow the instructions below to extract measurements:

In [ ]:
# (i) Create a dictionary that contains a key-value pairing for each measurement
# The keys should be a strings describing the type of measurement (e.g. 'cell_tubulin_mean') 
# and the values should be empty lists. These empty lists will be filled with the results of 
# the measurements and the dictionary will make it easy to work with this data.
results = {"cell_id":[],
           "cell_tubulin_mean":[],
           "cell_actin_mean":[],
           "cell_area":[]}

In [ ]:
# (ii) Record the measurements for each cell
# Iterate over the segmented cells (np.unique).
# Inside the loop, create a mask for the current cell and use it to extract the measurements listed above. 
# Add them to the appropriate list in the dictionary using the list.append method.
# Hint: Remember that you can get out all the values within a masked area by indexing the image 
#       with the mask. For example, np.mean(image[cell_mask]) will return the mean of all the 
#       intensity values of 'image' that are masked by 'cell_mask'.

# Iterate over cell IDs
for cell_id in np.unique(cell_seg)[1:]:

    # Mask the current cell
    cell_mask = cell_seg==cell_id
    
    # Get the measurements
    # Note: the .item() method ensures that the resulting number is converted from a numpy number object
    #       (e.g. type np.float) to a native python number object (e.g. type float). For most purposes,
    #       this is irrelevant, but for saving data in a python object as we do later on, it is useful
    #       to use native python objects only.
    results["cell_id"].append(cell_id.item())
    results["cell_area"].append(np.sum(cell_mask).item())
    results["cell_tubulin_mean"].append(np.mean(tritc[cell_mask]).item())
    results["cell_actin_mean"].append(np.mean(fitc[cell_mask]).item())

In [ ]:
# (iii) Print the results and check that they make sense
for key in results.keys(): print key, '\n', results[key], '\n'

# Writing output files

**Background**

The final step of the pipeline shows how to write output of the pipeline to files.

Data can be saved to files in a human-readable format such as text files (e.g. to import into Excel), in a format readable for other programs such as tif-images (e.g. to view in Fiji).

#### Exercices

Follow the instructions below

In [ ]:
# (i) Write one or more of the images you produced to a tif file
# Use the function imsave from the tifffile module. Make sure that the array you are writing
# is of integer type. If necessary, you can use the method 'img_as_uint'.
# After writing the file, load it into Fiji and check that everything worked as intended.
from tifffile import imsave
from skimage import img_as_uint

icomposite = img_as_uint(image= composite)
imsave("composite.tif",icomposite)

In [ ]:
# (ii) writing files to omero server
# - 1/ if you not already connected, connect you to omero server
# - 2/ get information on you
# - 3/ get project/dataset of mifobio group
# - 4/ write file to omero server

from omero.gateway import BlitzGateway
from numpy import array
import config

HOST = '10.5.0.156'
PORT = 4064
USERNAME = config.USERNAME
PASSWORD = config.PASSWORD
conn = BlitzGateway(USERNAME, PASSWORD, host=HOST, port=PORT)
conn.connect()
conn.setSecure(True)

user = conn.getUser()
print "Current user:"
print "  ID:", user.getId()
print "  Username:", user.getName()
print "  Full Name:", user.getFullName()
print "Member of:"
for g in conn.getGroupsMemberOf():
    print "  ID:", g.getName(), " Name:", g.getId()
group = conn.getGroupFromContext()
print "Current group: ", group.getName()

def print_obj(obj, indent=0):
    """
    Helper method to display info about OMERO objects.
    Not all objects will have a "name" or owner field.
    """
    print """%s%s:%s  Name:"%s" (owner=%s)""" % (
        " " * indent,
        obj.OMERO_CLASS,
        obj.getId(),
        obj.getName(),
        obj.getOwnerOmeName())

my_exp_id = conn.getUser().getId()

#switch group
conn.SERVICE_OPTS.setOmeroGroup(3)

for project in conn.getObjects("Project", opts={'owner': my_exp_id,
                                            'group': 3,
                                            'order_by': 'lower(obj.name)',
                                            'limit': 5, 'offset': 0}):
    print_obj(project)
    # We can get Datasets with listChildren, since we have the Project already.
    # Or conn.getObjects("Dataset", opts={'project', id}) if we have Project ID
    for dataset in project.listChildren():
        print_obj(dataset, 2)
        for image in dataset.listChildren():
            print_obj(image, 4)

#datasets = conn.getObjects("Dataset", opts={'Project': 2, 'group': 'mifobio'})
dataset = conn.getObject("Dataset", 3)

#for dataset in datasets:
size_x, size_y, size_z, size_c, size_t = 1024, 510, 1, 3, 1
plane1 = array(composite[:,:,0])
plane2 = array(composite[:,:,1])
plane3 = array(composite[:,:,2])
planes = [plane1, plane2, plane3]

def plane_gen():
    for p in planes:
        yield p

desc = "Image imported from a local image"
#print dataset

i = conn.createImageFromNumpySeq(
plane_gen(), "merge image", size_z, size_c, size_t, description=desc, dataset=dataset)

print 'Created new Image:%s Name:"%s"' % (i.getId(), i.getName())

In [ ]:
conn.close()

In [ ]:
# (v) Write a tab-separated text file of the results dict
# The most generic way of saving numerical results is a simple text file. It can be imported into 
# pretty much any other program.

# To write normal text files, open an empty file object in write mode ('w') using the 'with'-statement.
with open('cell_results.txt','w') as outfile:

    # Use the file_object.write(string) method to write strings to the file. First write the header of the
    # date (the result dict keys), separated by tabs ('\t'). It makes sense to first generate a complete
    # string with all the headers and then write this string to the file. Note that you will need to 
    # explicitly write 'newline' characters ('\n') at the end of the line to switch to the next line.
    sorted_keys = sorted(results.keys())
    header_string = '\t'.join(sorted_keys) + '\n'
    outfile.write(header_string)

    # After writing the headers, iterate over all the cells saved and write the data to the file by
    # creating strings similar to the header string.
    for index in range(len(results['cell_id'])):
        data_string = '\t'.join([str(results[key][index]) for key in sorted_keys]) + '\n'
        outfile.write(data_string)
        
# After writing the data, have a look at the output file in a text editor or in a spreadsheet
# program like Excel.

# Batch processing

Converting the pipeline to a function:

Convert the entire pipeline into a function that accepts a filename as input, runs everything, and returns the final segmentation and the results dictionary. To do this, you must:

    Add the function definition statement at the beginning of the script (after the imports)
    Replace the 'hard-coded' filename by a variable that is accepted by the function
    Indent all the code
    Add a return statement at the end


Importing the function and running it for multiple input files:

To actually run the pipeline function for multiple input files, we need to do the following:

    Iterate over all the filenames in a directory
    For each filename, call the pipeline function
    Collect the returned results

Follow the instructions in the code below.

In [ ]:
# (i) Test if your pipeline function actually works
# Import your function using the normal python syntax for imports, like this:
#   from your_module import your_function
# Run the function and visualize the resulting segmentation. Make sure everything
# works as intended.

# Import
from tuto_pipeline import run_pipeline

# Run
filename_1 = r'/home/cedric/Documents/Atelier_Mifobio/jupyter-workshop-mifobio-2018/exercices/PK-11B-pl1/Well-C003/FITC.jpeg'
filename_2 = r'/home/cedric/Documents/Atelier_Mifobio/jupyter-workshop-mifobio-2018/exercices/PK-11B-pl1/Well-C003/Hoechst.jpeg'
filename_3 = r'/home/cedric/Documents/Atelier_Mifobio/jupyter-workshop-mifobio-2018/exercices/PK-11B-pl1/Well-C003/Tritc.jpeg'

seg, results = run_pipeline(filename_1, filename_2, filename_3)

# Visualize
plt.figure(figsize=(7,7))
plt.imshow(np.zeros_like(seg),interpolation='none',cmap='gray',vmax=1)  # Black background
plt.imshow(np.ma.array(seg,mask=seg==0),interpolation='none',cmap='prism')
plt.show()

In [ ]:
# (ii) Get all relevant filenames from the directory
# Use the function 'walk' from the module 'os' to get a list of all the files
# in a directory.

# Get all files
from os import walk
target_files = []
wellnames = []
for root,_,filenames in walk("/home/cedric/Documents/Atelier_Mifobio/jupyter-workshop-mifobio-2018/exercices/PK-11B-pl1/"):
    for filename in filenames:
        wellnames.append(root)
        target_files.append(os.path.join(root, filename))

for x in wellnames:
    print x

In [ ]:
# (iii) Iterate over the relevant input filenames and run the pipeline function
# Be sure to collect the output of the pipeline function in a way that allows
# you to trace it back to the file it came from. You could for example use a
# dictionary with the wellnames as keys.
all_seg = {}
all_results = {}



for x in range(0,18,3):
    seg,results = run_pipeline(target_files[x+2],target_files[x+1], target_files[x+0])
    all_seg[wellnames[x]] = seg
    all_results[wellnames[x]] = results

# Visualize
plt.figure(figsize=(7,7))
plt.imshow(np.zeros_like(all_seg['/home/cedric/Documents/Atelier_Mifobio/jupyter-workshop-mifobio-2018/exercices/PK-11B-pl1/Well-A003']),interpolation='none',cmap='gray',vmax=1)  # Black background
plt.imshow(np.ma.array(all_seg['/home/cedric/Documents/Atelier_Mifobio/jupyter-workshop-mifobio-2018/exercices/PK-11B-pl1/Well-A003'],mask=all_seg['/home/cedric/Documents/Atelier_Mifobio/jupyter-workshop-mifobio-2018/exercices/PK-11B-pl1/Well-A003']==0),interpolation='none',cmap='prism')
plt.show()

In [ ]:
for key in all_results.keys(): print key, '\n', all_results[key], '\n'